In [1]:
# Imports

import pandas as pd
import numpy as np
import requests
import json
from bs4 import BeautifulSoup

from adquire_functions import get_students_and_katas
from cleaning_funcions import clean_katas
from chosing_functions import display_presenter_last_kata

In [2]:
def adquire(): 
    return get_students_and_katas()

def cleaning(df_students, df_katas): 
    return df_students, clean_katas(df_katas)

In [3]:
def get_completed_by_user(user, host='https://www.codewars.com/api/v1/users/', 
                          path='/code-challenges/completed'):
    url = host + user + path
    resp = requests.get(url)    
    return resp.json()

def user_time_kata(user, slug, completed, katas): 
    res = '2090-01-01T00:00:00.000Z'# default
    for data in completed[user]: 
        if data['slug'] == slug: 
            res = data['completedAt']
            break
    return res

def get_info_dict(df_students): 
    completed = {u: get_completed_by_user(u) for u in df_students.index}
    res_dict = dict( [(k, v['data']) for k, v in completed.items()] )
    return res_dict

def create_df_checking(df_students, df_katas): 
    katas = df_katas.index
    students = df_students.index
    
    # Creamos un DataFrame: Students x Katas
    df_res = pd.DataFrame(index=df_students.index, columns=df_katas.index)
    
    # tarda un rato
    completed_katas_dict = get_info_dict(df_students)
       
    # cada celda es el tiempo que ese student(row) tardó en realizar la kata(column)
    for user in students: 
        for slug in katas: 
            df_res.loc[user, slug] = user_time_kata(user, slug, completed_katas_dict, katas)
    
    # convertimos las columnas a tipo Datetime
    for col in df_res.columns: 
        df_res[col] = pd.to_datetime(df_res[col],infer_datetime_format=True)
        
    # En este punto df_res son valores temporales    
    # copiamos la solución en otro DF porque en el bucle la segunda vez da problemas al cambiar el tipo
    df_sol = df_res.copy()

    # display(df_res)
    for user in students: 
        df_sol.loc[user] = df_res.loc[user] < df_katas.limit
    
    return df_sol
    

In [4]:
def save_results(df, path='results.csv'): 
    df.to_csv(path, index=True)    

In [5]:
def color_negative_red(x):
    """
    Takes a boolean and returns a string with
    the css property `'color: red'` for False
    strings, green otherwise.
    """
    color = 'red' if x == False else 'green'
    return 'color: {}'.format(color) 

In [6]:
def main(): 
    df_students, df_katas = adquire()
    df_students, df_katas = cleaning(df_students, df_katas)
    df = create_df_checking(df_students, df_katas)
    
    # display(df)    
    save_results(df)
    s = df.style.applymap(color_negative_red)
    display(s)
    
    print( display_presenter_last_kata(df) )
    
    

    

In [7]:
if __name__ == '__main__':
    main()

slug,matrix-addition
username,
YonatanRA,True
alonsodani11,True
serguma,True
PatriciaRick,False
ricccardo,False
MartinML,True
iamjss,True
Gonzalo2104,True
Carlosdss,True


YonatanRA presentará la kata matrix-addition
